In [1]:
!pip install PyPDF2

# Import Libs

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import PyPDF2

# Load the OpenAI key

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


# Preparing System Prompt

In [4]:
system_prompt = "You are a helpful assistant that analyzes resumes and display a simple conslusion of the resume with some statistics on how it perfectly fits the job requirements and other informations. "
system_prompt = "Experiences in same domain add a huge advantage to the resume. "
system_prompt = "The user prompt should contains all information from the job requirements to the resume as a PDF format. "
system_prompt = "The response must include x short paragraphs: \n"
system_prompt = """
    1- How much out of 10 this user fits Job requirements
    2- Reason Behind the score
    3- what advices can you give to the HR of selecting this user or not
"""

# Extract Text from PDF

In [5]:
def get_pdf_text(path):
    text = ''
    
    with open(path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()

    return text

# AI Response Generator

In [6]:
def chat(requirements, file_path):
    file_text = get_pdf_text(file_path)
    message = f"Job Requirements: {requirements}\nResume Information: {file_text}"
    
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

# Create the Chat Interface

In [7]:
def create_resume_analyzer_interface():
    with gr.Blocks(title="Resume Analyzer 🤖") as interface:
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Requirements")
                input_text = gr.Textbox(
                    label = "Enter the Job Requirements",
                    placeholder = "Place your requirements here...",
                    lines = 7
                )
                file_upload = gr.File(
                    label="Upload Resume (PDF, DOC, DOCX, TXT)",
                    file_types=[".pdf", ".doc", ".docx", ".txt"],
                    file_count="single"
                )
            with gr.Column():
                gr.Markdown("### Result")
                output_text = gr.Markdown(
                    label="Analyzing...",
                    height=500  # Set height instead of lines
                )
        
                
        analyze_btn = gr.Button("Analyze", variant="primary")
        clear_btn = gr.Button("Clear", variant="secondary")
        
        analyze_btn.click(
            fn=chat,
            inputs=[input_text, file_upload],
            outputs=output_text
        )
        clear_btn.click(
            fn=lambda: ("", ""),
            inputs=[],
            outputs=[input_text, output_text]
        )   
    return interface

# Start Analyzing

In [8]:
interface = create_resume_analyzer_interface()

In [9]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
